In [ ]:
! pip3 install filetype

In [ ]:
import os
import cv2
import numpy as np 
from skimage.filters import threshold_yen
from skimage.exposure import rescale_intensity
from skimage.io import imread, imsave
import tensorflow as tf
import filetype
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import random
import ssl
import datetime
import sys
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Activation, Flatten, Dense, Dropout
from sklearn.model_selection import KFold 
from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [ ]:
train_data = [] 
train_label = []
count_folder = 0
def read_directory(directory_name,data,label,label_name):
    count = 0
    for filename in os.listdir(directory_name):
        if filename == '.DS_Store':
            continue
        count+=1
        img = tf.keras.preprocessing.image.load_img(directory_name + "/" + filename,target_size=(256,256,3))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = tf.keras.applications.densenet.preprocess_input(img)
        data.append(img)
        label.append(label_name)
    print(count)

In [ ]:
train_dir = '/content/drive/My Drive/gan'
def load_data(img_dir,data,label,label_number):
    for count,filename in enumerate(os.listdir(img_dir)):
        if filename == '.DS_Store':
                continue
        if os.path.isfile(img_dir + '/' + filename):
            print ("it's a normal file")
        else:
            print(filename+" it's a folder")
            for folder in range(6):
                if folder == '.DS_Store':
                    continue 
                print(img_dir+'/'+filename+"/"+str(label_number) + " : "+str(label_number))
                read_directory(img_dir+'/'+filename+"/"+str(label_number),data,label,label_number)
                label_number+= 1
    return label_number    

In [ ]:
count_folder = 0
count_folder = load_data(train_dir,train_data,train_label,count_folder)
print(count_folder)

train it's a folder
/content/drive/My Drive/gan/train/0 : 0
1174
/content/drive/My Drive/gan/train/1 : 1
992
/content/drive/My Drive/gan/train/2 : 2
600
/content/drive/My Drive/gan/train/3 : 3
871
/content/drive/My Drive/gan/train/4 : 4
739
/content/drive/My Drive/gan/train/5 : 5
1144
6


In [ ]:
train_label = tf.keras.utils.to_categorical(train_label, 6)

In [ ]:
train_data, train_label = shuffle(train_data, train_label, random_state=13)

In [ ]:
train_data = np.array(train_data)
train_label = np.array(train_label)

In [ ]:
inc = tf.keras.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3),
    pooling='max',
)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
def create_model():
    model = tf.keras.models.Sequential([
        inc,
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(6, activation='softmax')    
    ])
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model
model = create_model()
inc.summary(),model.summary()

74842112/74836368 [==============================] - 2s 0us/step
Model: "densenet201"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
_______________________

(None, None)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(vertical_flip=True,horizontal_flip=True)
LR_function = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5, verbose=1)
checkpoint = ModelCheckpoint("densenet_gan.h5", monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
kfold = KFold(10,True)
for train, valid in kfold.split(train_data):
  cross_train_data = train_data[train]
  cross_train_label = train_label[train]
  cross_valid_data = train_data[valid]
  cross_valid_label = train_label[valid]
  train_generator = train_gen.flow(train_data, train_label, batch_size = 8)
  model.fit_generator(
                train_generator,
                steps_per_epoch=len(cross_train_data)//8,
                epochs=10,
                verbose=1,
                validation_data = (cross_valid_data, cross_valid_label),
                callbacks=[LR_function,checkpoint])

In [ ]:
new_model = tf.keras.models.load_model('/content/densenet_gan.h5')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input
test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory=r"/content/drive/My Drive/aoi/test/",
    target_size=(200,200),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 10142 images belonging to 1 classes.


In [ ]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)
labels = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
import pandas as pd
filenames=test_generator.filenames

new_filenames=[]
for i in filenames:
  new_filenames.append(i[12:])

results=pd.DataFrame({"ID":new_filenames,
                      "Label":predictions})
results.to_csv("/content/drive/My Drive/dcgan_cross_densenet.csv",index=False)

Instructions for updating:
Please use Model.predict, which supports generators.
10142/10142 [==============================] - 7473s 737ms/step
